* Review Data in Table
* Implicit Index for Constraints
* Purpose of Index for Constraints
* Manage Indexes on Database Tables
* Relationship between Table and Indexes
* Indexes for Performance Tuning
* Exercise and Solution - Create Index on order date and order status
  * Index Name: orders_order_date_status_idx
  * Columns: `order_date` and `order_status`
  * Once index is created, make sure to review the explain plans for the below queries.

```sql
SELECT count(*)
FROM orders
WHERE order_date = '2014-01-01 00:00:00.0';

SELECT count(*)
FROM orders
WHERE order_date = '2014-01-01 00:00:00.0'
AND order_status = 'COMPLETE';

SELECT count(*)
FROM orders
WHERE order_date = '2014-01-01 00:00:00.0'
AND order_status = 'CLOSED';
```

* Review Data in Table

```sql
CREATE TABLE users (
	id INT,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	username VARCHAR(50),
	email VARCHAR(50)
);

INSERT INTO users
VALUES
	(1, 'Durga', 'Gadiraju', 'dgadiraju', 'dgadiraju@email.com'),
	(4, 'Scott', 'Tiger', 'stiger', 'stiger@email.com'),
	(2, 'Mickey', 'Mouse', 'mmouse', 'mmouse@email.com'),
	(5, 'Elvis', 'Presley', 'epresley', 'epresley@email.com'),
	(3, 'Charlie', 'Chaplin', 'cchaplin', 'cchaplin@email.com');
	
SELECT ctid, u.* FROM users AS u;

EXPLAIN
SELECT * FROM users
WHERE id = 4;
```

* Overview of Database Object Metadata

Here are the details about Database Object Metadata (using Postgres as reference)
* A Database contains different types of objects - tables, views, stored procedures, etc.
* A Database Table contain columns, data types, constraints, indexes, etc.
* The details about the table such as columns, data types, constraints, indexes, etc is known as metadata.
* The metadata is typically stored in internal schemas or databases. For postgres database tables, the metadata is typically stored in a special schema called as `information_schema`.
* The schema contains tables and views to expose all the metadata related to database tables and other objects.
* Here are some of the important tables/views.
  * `information_schema.tables`
  * `information_schema.constraints`
  * `pg_stat_all_indexes` - not in information_schema

```sql
SELECT * FROM information_schema.tables
WHERE table_schema = 'public';

SELECT * FROM information_schema.columns
WHERE table_schema = 'public' AND table_name = 'users';

SELECT * FROM information_schema.constraint_column_usage
WHERE table_schema = 'public' AND table_name = 'orders';

SELECT * FROM pg_stat_all_indexes
WHERE schemaname = 'public'
	AND relname = 'users';
```

* Implicit Index for Constraints

1. When it comes to database tables, the columns can be defined using different types of constraints. Not null, foreign key, unique, and primary key.
2. Unique and Primary key constraint columns are always indexed.
3. We cannot drop the indexes that are implicitly created for unique and primary key constraints.

```sql
DROP TABLE users;

CREATE TABLE users (
	id INT PRIMARY KEY,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	username VARCHAR(50) UNIQUE,
	email VARCHAR(50)
);

INSERT INTO users
VALUES
	(1, 'Durga', 'Gadiraju', 'dgadiraju', 'dgadiraju@email.com'),
	(4, 'Scott', 'Tiger', 'stiger', 'stiger@email.com'),
	(2, 'Mickey', 'Mouse', 'mmouse', 'mmouse@email.com'),
	(5, 'Elvis', 'Presley', 'epresley', 'epresley@email.com'),
	(3, 'Charlie', 'Chaplin', 'cchaplin', 'cchaplin@email.com');

SELECT * FROM information_schema.constraint_column_usage
WHERE table_schema = 'public' AND table_name = 'users';

SELECT * FROM pg_stat_all_indexes
WHERE schemaname = 'public'
	AND relname = 'users';

DROP INDEX users_pkey; -- The command will fail
```

* Purpose of Index for Constraints

Here is the pseudo code that is used to explain the purpose of index for constraints.

```text
On insert:
    Lookup for id in the table
    If id exists then fail else proceed
    Lookup for username in the table
    If username exists then fail else proceed
    Proceed and insert data into the table
```

* Manage Indexes on Database Tables

1. Drop users table - `DROP TABLE users`
2. Create users table with only primary key on id

```sql
DROP TABLE users;

CREATE TABLE users (
	id INT PRIMARY KEY,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	username VARCHAR(50),
	email VARCHAR(50)
);

INSERT INTO users
VALUES
	(1, 'Durga', 'Gadiraju', 'dgadiraju', 'dgadiraju@email.com'),
	(4, 'Scott', 'Tiger', 'stiger', 'stiger@email.com'),
	(2, 'Mickey', 'Mouse', 'mmouse', 'mmouse@email.com'),
	(5, 'Elvis', 'Presley', 'epresley', 'epresley@email.com'),
	(3, 'Charlie', 'Chaplin', 'cchaplin', 'cchaplin@email.com');

SELECT * FROM information_schema.constraint_column_usage
WHERE table_schema = 'public' AND table_name = 'users';

SELECT * FROM pg_stat_all_indexes
WHERE schemaname = 'public'
	AND relname = 'users';
```

3. Create index on username column of users table

```sql
CREATE INDEX users_username_idx
ON users (username);

SELECT * FROM pg_stat_all_indexes
WHERE schemaname = 'public'
	AND relname = 'users';

-- Index can also be unique
DROP INDEX users_username_idx;

CREATE UNIQUE INDEX users_username_idx
ON users (username); -- if existing values in username are not unique, then this will fail

SELECT * FROM pg_stat_all_indexes
WHERE schemaname = 'public'
	AND relname = 'users';
```

4. Review Explain Plans on Queries using username.

```sql
EXPLAIN
SELECT * FROM users 
WHERE username = 'stiger';
```

* Relationship between Table and Indexes

1. There can be more than one index on a given table.
2. Implicit indexes will be created for each primary key and unique constraints.
3. Additional indexes can be added based on the need. An index can be normal index or unique.
4. If we drop the table, all the indexes will be automatically dropped.
5. Index will have data separately from the table. It contain the actual index column values along with corresponding rowid/ctid in the table.

* Indexes for Performance Tuning

1. Check the queries used by the application
2. If a particular column is used in the application very often, we can consider adding the index.
3. For example, we might end up searching users based on user email, hence we can consider adding index on email field in users table.
4. Use `CREATE INDEX` command to create index and test the application to see if the performance is improved or not.

```sql
CREATE TABLE users (
	id INT PRIMARY KEY,
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	username VARCHAR(50),
	email VARCHAR(50)
);

INSERT INTO users
VALUES
	(1, 'Durga', 'Gadiraju', 'dgadiraju', 'dgadiraju@email.com'),
	(4, 'Scott', 'Tiger', 'stiger', 'stiger@email.com'),
	(2, 'Mickey', 'Mouse', 'mmouse', 'mmouse@email.com'),
	(5, 'Elvis', 'Presley', 'epresley', 'epresley@email.com'),
	(3, 'Charlie', 'Chaplin', 'cchaplin', 'cchaplin@email.com');

SELECT * FROM information_schema.constraint_column_usage
WHERE table_schema = 'public' AND table_name = 'users';

SELECT * FROM pg_stat_all_indexes
WHERE schemaname = 'public'
	AND relname = 'users';

CREATE UNIQUE INDEX users_email_idx
ON users (email);

EXPLAIN
SELECT * FROM users
WHERE email = 'dgadiraju@email.com';
```

5. We can also add indexes to fine tune the performance of joins. For example, we would like to get all the orders placed by a given customer (using customer_id). With out index on orders customer id, we might end up with full table scan for each request.

```sql
SELECT count(*) FROM customers;
SELECT count(*) FROM orders;

SELECT c.customer_id, 
	c.customer_email,
	o.order_id,
	o.order_date,
	o.order_status
FROM customers AS c
	JOIN orders AS o
		ON c.customer_id = o.order_customer_id
WHERE c.customer_id = 2;

EXPLAIN
SELECT c.customer_id, 
	c.customer_email,
	o.order_id,
	o.order_date,
	o.order_status
FROM customers AS c
	JOIN orders AS o
		ON c.customer_id = o.order_customer_id
WHERE c.customer_id = 2;

CREATE INDEX orders_order_customer_id_idx
ON orders (order_customer_id);

EXPLAIN
SELECT c.customer_id, 
	c.customer_email,
	o.order_id,
	o.order_date,
	o.order_status
FROM customers AS c
	JOIN orders AS o
		ON c.customer_id = o.order_customer_id
WHERE c.customer_id = 2;
```